# 特定物体認識

狭義の物体認識は，入力データに対してラベルを出力する識別タスクを指すことが多いです．
ラベルは，必要に応じてあらかじめ人間が決めておきます．
たとえば，「コップ」や「車」等，物体の一般名称を指すラベルが多く使われます．
あるいは，たくさんの3Dモデルデータを集めたデータベースが存在する場合に，入力データの物体と一致するモデルを特定する（ID等を返す）ことも物体認識タスクの一つです．
前者がカテゴリレベルの認識タスク，すなわち一般物体認識と呼ばれるのに対し，後者はインスタンスレベルの認識，すなわち特定物体認識と呼ばれます．
いずれにしても，（識別タスクを指すところの）物体認識は以下の手順で実行されます．

1. ラベルの付与された3Dデータを用意する
2. 3Dデータから特徴量を抽出する
3. 特徴量からラベルを推定する識別器を用意（学習）する
4. 認識対象物体の3Dデータから特徴量を抽出する
5. 識別器を用いてラベルを推定する

早速ですが，実際の3次元点群データを用いて物体認識を実行してみましょう．
ここでは，思いつく限り最も簡単な例と手法を試してみます．
まず，下記を実行して「手順１：ラベルの付与された3Dデータを用意する」を行います．

In [1]:
import os

dirname = "rgbd-dataset"
classes = ["apple", "banana", "camera"]
url="https://rgbd-dataset.cs.washington.edu/dataset/rgbd-dataset_pcd_ascii/"
for i in range(len(classes)):
    if not os.path.exists(dirname + "/" + classes[i]):
        os.system("wget " + url + classes[i] + "_1.tar")
        os.system("tar xvf " + classes[i] + "_1.tar")

これを実行すると有名な3D点群データセットRGB-D Object Datasetの一部のデータがダウンロードされます．
RGB-D Object Datasetは，2011年にKevin Laiらによって公開されたベンチマークデータセットで，
2010年に発売開始されたMicrosoft Kinectセンサのプロトタイプを用いて撮影された，全51カテゴリ300個の物体の3Dデータからなるデータセットです．
3Dデータとして，RGBD画像およびPCD形式の点群データが提供されていますが，今回は後者の点群データを扱います．
なお，データセット全体の容量が大きいため，今回は*apple*，*banana*，*camera*カテゴリから各一個ずつのインスタンスのみをダウンロードして用います．

次に，「手順２：3Dデータから特徴量を抽出する」から「手順４：認識対象物体の3Dデータから特徴量を抽出する」までを行います．
今回の例では，「手順３：特徴量からラベルを推定する識別器を用意（学習）する」を簡略化し，$k=1$の$k$最近傍法を用いることにします．
すなわち，認識対象物体の特徴量とデータベース内の全物体の特徴量の類似度を計算し，最も類似度の高いデータベース物体のラベルを出力するという戦略です．
まずは，特徴抽出関数を定義しましょう．

In [2]:
import open3d as o3d
import numpy as np

def extract_fpfh( filename ):
    print (" ", filename)
    pcd = o3d.io.read_point_cloud(filename)
    pcd = pcd.voxel_down_sample(0.01)
    pcd.estimate_normals(
        search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.02, max_nn=10))
    fpfh = o3d.pipelines.registration.compute_fpfh_feature(pcd,
        search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.03, max_nn=100))
    sum_fpfh = np.sum(np.array(fpfh.data),1)
    return( sum_fpfh / np.linalg.norm(sum_fpfh) )

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


関数`extract_fpfh`はファイルから点群データを読み込み，ダウンサンプリングと法線ベクトル推定を行った後に，
各点からFPFH特徴量を抽出します．
FPFH特徴量は33次元の局所特徴量ですが，
今回は，点群データ全体を一つの物体として認識するために，点群データ全体から大域特徴量を抽出する必要があります．
そこで，（最も単純な方法ではありますが）12行目で全ての点から抽出されるFPFH特徴量を総和し，
13行目で特徴量のノルムを$1$に正規化して返します．

さて，先にダウンロードした3個の物体の点群データに対して特定物体認識を実行してみましょう．
先のシェルスクリプトを実行すると，`rgbd-dataset/`フォルダ以下に，*apple_1*，*banana_1*，*camera_1*の各物体につき$600$個以上の点群データが格納されます．
これらの点群データは全て同じ物体を異なる角度（姿勢）で撮影したデータです．
データ撮影には回転台が使われており，方位角（Azimuth）を細かく変化させるだけでなく，３パターンの高度角（Elevation）で撮影されています．
各点群データのファイル名は，たとえば`apple_1_x_y.pcd`といったものになっており，$x$が高度角のレベルを，$y$が方位角のレベルを示しています．
下記のコードは，これらの点群データのうち$100$個を学習データ，別の$100$個をテストデータとして読み込み，特徴量を抽出します．

In [3]:
nsamp = 100
feat_train = np.zeros( (len(classes), nsamp, 33) )
feat_test = np.zeros( (len(classes), nsamp, 33) )

for i in range(len(classes)):
    print ("Extracting train features in " + classes[i] + "...")
    for n in range(nsamp):
        filename = dirname + "/" + classes[i] + "/" + classes[i] + \
                   "_1/" + classes[i] + "_1_1_" + str(n+1) + ".pcd"
        feat_train[ i, n ] = extract_fpfh( filename )
    print ("Extracting test features in " + classes[i] + "...")
    for n in range(nsamp):
        filename = dirname + "/" + classes[i] + "/" + classes[i] + \
                   "_1/" + classes[i] + "_1_4_" + str(n+1) + ".pcd"
        feat_test[ i, n ] = extract_fpfh( filename )

Extracting train features in apple...
  rgbd-dataset/apple/apple_1/apple_1_1_1.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_2.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_3.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_4.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_5.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_6.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_7.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_8.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_9.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_10.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_11.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_12.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_13.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_14.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_15.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_16.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_17.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_18.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_19.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_20.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_21.pcd
  rgb

  rgbd-dataset/apple/apple_1/apple_1_4_90.pcd
  rgbd-dataset/apple/apple_1/apple_1_4_91.pcd
  rgbd-dataset/apple/apple_1/apple_1_4_92.pcd
  rgbd-dataset/apple/apple_1/apple_1_4_93.pcd
  rgbd-dataset/apple/apple_1/apple_1_4_94.pcd
  rgbd-dataset/apple/apple_1/apple_1_4_95.pcd
  rgbd-dataset/apple/apple_1/apple_1_4_96.pcd
  rgbd-dataset/apple/apple_1/apple_1_4_97.pcd
  rgbd-dataset/apple/apple_1/apple_1_4_98.pcd
  rgbd-dataset/apple/apple_1/apple_1_4_99.pcd
  rgbd-dataset/apple/apple_1/apple_1_4_100.pcd
Extracting train features in banana...
  rgbd-dataset/banana/banana_1/banana_1_1_1.pcd
  rgbd-dataset/banana/banana_1/banana_1_1_2.pcd
  rgbd-dataset/banana/banana_1/banana_1_1_3.pcd
  rgbd-dataset/banana/banana_1/banana_1_1_4.pcd
  rgbd-dataset/banana/banana_1/banana_1_1_5.pcd
  rgbd-dataset/banana/banana_1/banana_1_1_6.pcd
  rgbd-dataset/banana/banana_1/banana_1_1_7.pcd
  rgbd-dataset/banana/banana_1/banana_1_1_8.pcd
  rgbd-dataset/banana/banana_1/banana_1_1_9.pcd
  rgbd-dataset/banana/

  rgbd-dataset/banana/banana_1/banana_1_4_59.pcd
  rgbd-dataset/banana/banana_1/banana_1_4_60.pcd
  rgbd-dataset/banana/banana_1/banana_1_4_61.pcd
  rgbd-dataset/banana/banana_1/banana_1_4_62.pcd
  rgbd-dataset/banana/banana_1/banana_1_4_63.pcd
  rgbd-dataset/banana/banana_1/banana_1_4_64.pcd
  rgbd-dataset/banana/banana_1/banana_1_4_65.pcd
  rgbd-dataset/banana/banana_1/banana_1_4_66.pcd
  rgbd-dataset/banana/banana_1/banana_1_4_67.pcd
  rgbd-dataset/banana/banana_1/banana_1_4_68.pcd
  rgbd-dataset/banana/banana_1/banana_1_4_69.pcd
  rgbd-dataset/banana/banana_1/banana_1_4_70.pcd
  rgbd-dataset/banana/banana_1/banana_1_4_71.pcd
  rgbd-dataset/banana/banana_1/banana_1_4_72.pcd
  rgbd-dataset/banana/banana_1/banana_1_4_73.pcd
  rgbd-dataset/banana/banana_1/banana_1_4_74.pcd
  rgbd-dataset/banana/banana_1/banana_1_4_75.pcd
  rgbd-dataset/banana/banana_1/banana_1_4_76.pcd
  rgbd-dataset/banana/banana_1/banana_1_4_77.pcd
  rgbd-dataset/banana/banana_1/banana_1_4_78.pcd
  rgbd-dataset/banan

  rgbd-dataset/camera/camera_1/camera_1_4_45.pcd
  rgbd-dataset/camera/camera_1/camera_1_4_46.pcd
  rgbd-dataset/camera/camera_1/camera_1_4_47.pcd
  rgbd-dataset/camera/camera_1/camera_1_4_48.pcd
  rgbd-dataset/camera/camera_1/camera_1_4_49.pcd
  rgbd-dataset/camera/camera_1/camera_1_4_50.pcd
  rgbd-dataset/camera/camera_1/camera_1_4_51.pcd
  rgbd-dataset/camera/camera_1/camera_1_4_52.pcd
  rgbd-dataset/camera/camera_1/camera_1_4_53.pcd
  rgbd-dataset/camera/camera_1/camera_1_4_54.pcd
  rgbd-dataset/camera/camera_1/camera_1_4_55.pcd
  rgbd-dataset/camera/camera_1/camera_1_4_56.pcd
  rgbd-dataset/camera/camera_1/camera_1_4_57.pcd
  rgbd-dataset/camera/camera_1/camera_1_4_58.pcd
  rgbd-dataset/camera/camera_1/camera_1_4_59.pcd
  rgbd-dataset/camera/camera_1/camera_1_4_60.pcd
  rgbd-dataset/camera/camera_1/camera_1_4_61.pcd
  rgbd-dataset/camera/camera_1/camera_1_4_62.pcd
  rgbd-dataset/camera/camera_1/camera_1_4_63.pcd
  rgbd-dataset/camera/camera_1/camera_1_4_64.pcd
  rgbd-dataset/camer

ここでは，高度角レベルが$1$のデータを学習データとし，高度角レベルが$4$のデータをテストデータとしています．
学習データおよびテストデータの特徴量は，それぞれ10行目と15行目で`feat_train`および`feat_test`変数に格納されます．

最後に，「手順５：識別器を用いてラベルを推定する」を行います．
前述のとおり，ここでは最も単純な方法である$k=1$の$k$最近傍法を用います．
学習データの点群データもテストデータの点群データも，それぞれが同様に$33$次元の特徴ベクトルで表されています．
今回は，二つの点群データの類似度としてベクトルの内積を用います．
以上の処理を行うコードを下記に示します．

In [4]:
for i in range(len(classes)):
    max_sim = np.zeros((3, nsamp))
    for j in range(len(classes)):
        sim = np.dot(feat_test[i], feat_train[j].transpose())
        max_sim[j] = np.max(sim,1)
    correct_num = (np.argmax(max_sim,0) == i).sum()
    print ("Accuracy of", classes[i], ":", correct_num*100/nsamp, "%")

Accuracy of apple : 98.0 %
Accuracy of banana : 89.0 %
Accuracy of camera : 83.0 %


変数`max_sim`は，全てのテストデータに対して，5行目で，
$j$番目の物体の全学習データの中で最も近いデータとの類似度を格納しています．
この類似度が最も高いクラスが推定されたラベルとなります．
6行目では，各テストデータの推定ラベルが，そのテストデータ本来のラベル（$i$）に一致する場合の個数を計算し，
7行目でその結果を標準出力に出しています．